# Topology in time series forecasting: `TransformerResamplerMixin`s and `giotto-tda` `Pipeline`s

This notebook explains how to use `giotto-tda` to create interesting topological features that can help in time series forecasting tasks, and how to integrate them into `scikit-learn`–compatible pipelines.

In particular, we will concentrate on topological features which are created from consecutive **sliding windows** over the data. In sliding window models, a single time series `X` originally encoded as an array of shape ``(n_timestamps, n_features)`` is turned into a time series of windows over the data, with a new shape ``(n_windows, n_samples_per_window, n_features)``. Two issues arise:
1. ``n_windows`` is smaller than ``n_timestamps``. This is because we cannot have more windows than there are time stamps without padding `X`, and this is not done by `giotto-tda`. ``n_timestamps - n_windows`` is even larger if we decide to pick a large stride between consecutive windows. 
2. The target variable `y` needs to be properly "aligned" with each window so that the forecasting problem is meaningful and e.g. we don't "leak" information from the future. In particular, `y` needs to be "resampled" so that it too has length `n_windows`.

To deal with these issues, `giotto-tda` provides a selection of relevant transformers with `resample`, `transform_resample` and `fit_transform_resample` methods. These are inherited from a `TransformerResamplerMixin` base class. Furthermore, `giotto-tda` provides a drop-in replacement for `scikit-learn`'s `Pipeline` which extends it to allow chaining `TransformerResamplerMixin`s with regular `scikit-learn` estimators.

If you are looking at a static version of this notebook and would like to run its contents, head over to [GitHub](https://github.com/giotto-ai/giotto-tda/blob/master/examples/time_series_forecasting.ipynb) and download the source.

**License: AGPLv3**

## `SlidingWindow`

Let us start with a simple example of a "time series" `X` with a corresponding target `y` of the same length.

In [ ]:
import numpy as np

n_timestamps = 10
X, y = np.arange(n_timestamps), np.arange(n_timestamps) - n_timestamps
X, y

We can instantiate our sliding window transformer-resampler and run it on the pair `(X, y)`:

In [ ]:
from gtda.time_series import SlidingWindow

window_size = 3
stride = 2

SW = SlidingWindow(size=window_size, stride=stride)
Xt, yr = SW.fit_transform_resample(X, y)
Xt, yr

We note a couple of things:
- `fit_transform_resample` returns a pair: the window-transformed `X` and the resampled and aligned `y`.
- `SlidingWindow` has made a choice for us on how to resample `y` and line it up with the windows from `X`: a window on `X` corredponds to the *last* value in a corresponding window over `y`. This is common in time series forecasting where, for example, `y` could be a shift of `X` by one time stamp.
- Some of the initial values of `X` may not be found in `Xt`. This is because `SlidingWindow` only ensures the *last* value is represented in the last window, regardless of the stride. 

## Multivariate time series example: Sliding window + topology `Pipeline`

`giotto-tda`'s topology transformers expect 3D input. But our `Xt` above is 2D. How do we capture interesting properties of the topology of input time series then? For univariate time series, it turns out that a good way is to use the "time delay embedding" or "Takens embedding" technique explained in the first part of [Topology of time series](https://github.com/giotto-ai/giotto-tda/blob/master/examples/topological_timeseries.ipynb). But as this involves an extra layer of complexity, we leave it to later and concentrate for now on an example with a simpler API which also demonstrates the use of a `giotto-tda` `Pipeline`.

Surprisingly, this involves multivariate time series input!

In [ ]:
rng = np.random.default_rng(42)

n_features = 2

X = rng.integers(0, high=20, size=(n_timestamps, n_features), dtype=int)
X

We are interpreting this input as a time series in two variables, of length `n_timestamps`. The target variable is the same `y` as before.

In [ ]:
SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X, y)
X_sw, yr

`Xt` is now a complicated-looking array, but it has a simple interpretation. Again, `Xt[i]` is the `i`-th window on `X`, and it contains `window_size` samples from the original time series. This time, the samples are not scalars but 1D arrays.

What if we suspect that the way in which the **correlations** between the variables evolve over time can help forecast the target `y`? This is a common situation in neuroscience, where each variable could be data from a single EEG sensor, for instance.

`giotto-tda` exposes a `PearsonDissimilarity` transformer which creates a 2D dissimilarity matrix from each window in `Xt`, and stacks them together into a single 3D object. This is the correct format (and information content!) for a typical topological transformer in `gtda.homology`. See also [Topological feature extraction from graphs](https://github.com/giotto-ai/giotto-tda/blob/master/examples/persistent_homology_graphs.ipynb) for an in-depth look. Finally, we can extract simple scalar features using a selection of transformers in `gtda.diagrams`.

In [ ]:
from gtda.time_series import PearsonDissimilarity
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import Amplitude

PD = PearsonDissimilarity(); X_pd = PD.fit_transform(X_sw)
VR = VietorisRipsPersistence(metric="precomputed"); X_vr = VR.fit_transform(X_pd)  # "precomputed" required on dissimilarity data
Ampl = Amplitude(); X_a = Ampl.fit_transform(X_vr)
X_a

Notice that we are not acting on `y` above. We are simply creating features from each window using topology! *Note*: it's two features per window because we used the default value for `homology_dimensions` in `VietorisRipsPersistence`, not because we had two variables in the time series initially!

We can now put this all together into a `giotto-tda` `Pipeline` which accommodates for both the initial sliding window transformation on `X`/resampling of `y`, and the feature extraction from the windows on `X`. *Note*: while we could import the `Pipeline` class and use its constructor, we use the convenience function `make_pipeline` instead, which is a drop-in replacement for [`scikit-learn`'s](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html).

In [ ]:
from gtda.pipeline import make_pipeline

pipe = make_pipeline(SW, PD, VR, Ampl)
pipe.fit_transform_resample(X, y)

Finally, if we have a *regression* task on `y` we can add a final estimator such as scikit-learn's `RandomForestRegressor` as a final step in the previous pipeline, and fit it!

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RFR = RandomForestRegressor()

pipe = make_pipeline(SW, PD, VR, Ampl, RFR)
pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

## Univariate time series – `TakensEmbedding` and `MultiTakensEmbedding`

The first part of [Topology of time series](https://github.com/giotto-ai/giotto-tda/blob/master/examples/topological_timeseries.ipynb) explains a commonly used technique for converting a univariate time series into a single **point cloud**. Since topological features can be extracted from any point cloud, this is a gateway to time series analysis using topology. The second part of that notebook shows how to transform a *batch* of time series into a batch of point clouds, and how to extract topological descriptors from each of them independently. While in that notebook this is applied to a time series classification task is time series classification, in this notebook we are concerned with topology-powered *forecasting* from a single time series.

Reasoning by analogy with the multivariate case above, we can look at sliding windows over `X` as small time series in their own right and track the evolution of *their* topology against the variable of interest (or against itself, if we are interested in unsupervised tasks such as anomaly detection).

There are two ways in which we can implement this idea in `giotto-tda`:
1. We can *first* compute a global Takens embedding of the time series, which takes us from 1D/column data to 2D data, and *then* partition the 2D data of vectors into sliding windows via `SlidingWindow`.
2. We can first apply a `SlidingWindow`, and then an instance of `MultiTakensEmbedding`.

The first route seems upside-down, but it has the advantange that globally "optimal" parameters for the "time delay" and "embedding dimension" parameters can be computed automatically by `TakensEmbedding`. The second route, however, ensures that we can run our "topological feature extraction track" in parallel with other feature-generation pipelines from sliding windows, without experiencing shape mismatches.

Below is what each route would look like. *Remark:* In the presence of noise, a small sliding window size is likely to reduce the reliability of the estimate of the time serie's local topology.

### Option 1: `TakensEmbeding` + `SlidingWindow`

Note that `TakensEmbedding` is also a `TransformerResamplerMixin`, and that the logic for resampling/aligning `y` is the same as in `SlidingWindow`.

In [ ]:
from gtda.time_series import TakensEmbedding

X = np.arange(n_timestamps)

TE = TakensEmbedding(parameters_type="search", time_delay=2, dimension=3)
X_te, yr = TE.fit_transform_resample(X, y)
X_te, yr

In [ ]:
window_size = 5
stride = 2

SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X_te, yr)
X_sw, yr

From here on, it is easy to push a very similar pipeline through as in the multivariate case:

In [ ]:
VR = VietorisRipsPersistence()  # No "precomputed" for point clouds
Ampl = Amplitude()
RFR = RandomForestRegressor()

pipe = make_pipeline(TE, SW, VR, Ampl, RFR)

pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

### Option 2: `SlidingWindow` + `MultiTakensEmbedding`

`MultiTakensEmbedding` is not a `TransformerResamplerMixin`, but this is not a problem in the context of a `Pipeline` when we order things in this way.

In [ ]:
from gtda.time_series import MultiTakensEmbedding

X = np.arange(n_timestamps)

window_size = 5
stride = 2

SW = SlidingWindow(size=window_size, stride=stride)
X_sw, yr = SW.fit_transform_resample(X, y)
X_sw, yr

In [ ]:
time_delay = 1
dimension = 2

MTE = MultiTakensEmbedding(time_delay=time_delay, dimension=dimension)
X_mte = MTE.fit_transform(X_sw)
X_mte

In [ ]:
VR = VietorisRipsPersistence()  # No "precomputed" for point clouds
Ampl = Amplitude()
RFR = RandomForestRegressor()

pipe = make_pipeline(SW, MTE, VR, Ampl, RFR)
pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

### Integrating non-topological features

The best results are obtained when topological methods are used not in isolation but in **combination** with other methods. Here's an example where, in parallel with the topological feature extraction from local sliding windows using **Option 2** above, we also compute the mean and variance in each sliding window. A `scikit-learn` `FeatureUnion` is used to combine these very different sets of features into a single pipeline object.

In [ ]:
from functools import partial
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_union
from sklearn.base import clone

mean = FunctionTransformer(partial(np.mean, axis=1, keepdims=True))
var = FunctionTransformer(partial(np.var, axis=1, keepdims=True))

pipe_topology = make_pipeline(MTE, VR, Ampl)

feature_union = make_union(mean, var, pipe_topology)
    
pipe = make_pipeline(SW, feature_union, RFR)

pipe.fit(X, y)
y_pred = pipe.predict(X)
score = pipe.score(X, y)
y_pred, score

## Endogeneous target preparation with `Labeller`

Let us say that we simply wish to predict the future of a time series from itself. This is very common in the study of financial markets for example. `giotto-tda` provides convenience classes for target preparation from a time series. This notebook only shows a very simple example: many more options are described in `Labeller`'s documentation.

If we wished to create a target `y` from `X` such that `y[i]` is equal to `X[i + 1]`, while also modifying `X` and `y` so that they still have the same length, we could proceed as follows:

In [ ]:
from gtda.time_series import Labeller

X = np.arange(10)

Lab = Labeller(size=1, func=np.max)
Xl, yl = Lab.fit_transform_resample(X, X)
Xl, yl

Notice that we are feeding two copies of `X` to `fit_transform_resample` in this case!

This is what fitting an end-to-end pipeline for future prediction using topology could look like. Again, you are encouraged to include your own non-topological features in the mix!

In [ ]:
TE = TakensEmbedding()
SW = SlidingWindow(size=5)
VR = VietorisRipsPersistence()
Ampl = Amplitude()
RFR = RandomForestRegressor()

# Pipeline up to the regressor, to inspect the resampling
pipe_ = make_pipeline(Lab, TE, SW, VR, Ampl)

# Full pipeline including the regressor
pipe = make_pipeline(Lab, TE, SW, VR, Ampl, RFR)

pipe.fit(X, X)
pipe.predict(X)

## Where to next?

1. There are two additoinal simple `TransformerResamplerMixin`s in `gtda.time_series`: `Resampler` and `Stationarizer`.
2. The sort of pipeline for topological feature extraction using Takens embedding is a bit crude. More sophisticated methods exist for extracting robust topological summaries from (windows on) time series. A good source of inspiration is the following paper:
[Persistent Homology of Complex Networks for Dynamic State Detection](https://arxiv.org/abs/1904.07403), by A. Myers, E. Munch, and F. A. Khasawneh
The module `gtda.graphs` contains several transformers implementing the main algorithms proposed there.
3. Advanced users may be interested in `ConsecutiveRescaling`, which can be found in `gtda.point_clouds`.
4. The notebook [Case study: Lorenz attractor](https://github.com/giotto-ai/giotto-tda/blob/master/examples/lorenz_attractor.ipynb) is an advanced use-case for `TakensEmbedding` and other time series forecasting techniques inspired by topology.